# Example Usage of PairedData Objects

### Setup

In [1]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))

import json
from typing import Sequence, cast

import numpy as np

from hec import DssDataStore, TimeSeries, UsgsRounder
from hec.rating import PairedData

dss_file_name = "../test/resources/rating/Paired_Data.dss"
assert os.path.exists(dss_file_name)
DssDataStore.set_message_level(1)
rounder = UsgsRounder("9999999993")

### Use a PairedData Object with 1 Indpendent Parameter and 1 Dependent Parameter

In [2]:
with DssDataStore.open(dss_file_name) as dss:
    pd = cast(
        PairedData, dss.retrieve("/MARK_TWAIN_LAKE/TAINTER GATES-RATING/ELEV-FLOW////")
    )
    elevs = pd.ind_values
    flows = pd.dep_values[0]
    # ------------------- #
    # rate a single value #
    # ------------------- #
    idx = int(len(elevs) / 2)
    elev = elevs[idx]
    rated_float = pd.rate(elev)
    assert rated_float == flows[idx]
    print(f"Float: Elev {elev} -> Flow {rated_float}")
    # now reverse rate
    reverse_rated_float = pd.reverse_rate(rated_float)
    assert reverse_rated_float == elev
    # --------------------- #
    # rate a list of values #
    # --------------------- #
    rated_sequence = pd.rate(elevs)
    assert np.allclose(rated_sequence, flows, equal_nan=True)
    print(f"Sequence of Floats: (Elev -> Flow) = {list(zip(elevs, rated_sequence))}")
    # now reverse rate
    reverse_rated_sequence = pd.reverse_rate(rated_sequence)
    assert np.allclose(reverse_rated_sequence, elevs, equal_nan=True)
    # ------------------ #
    # rate a time series #
    # ------------------ #
    elev_ts = cast(
        TimeSeries,
        dss.retrieve("/MARK_TWAIN_LAKE/TAINTER GATES-RATING/ELEV//1Hour/Test/"),
    )
    flow_ts = cast(
        TimeSeries,
        dss.retrieve("/MARK_TWAIN_LAKE/TAINTER GATES-RATING/FLOW//1Hour/Test/"),
    )
    rated_ts = pd.rate(elev_ts)
    # don't compare first and last value in time series because they are out of bounds (Java extrapolates and python set to NaN)
    assert np.allclose(rated_ts.values[1:-1], flow_ts.values[1:-1], equal_nan=True)
    print(f"TimeSeries: (Elev -> Flow) = {list(zip(elev_ts.values, rated_ts.values))}")

Float: Elev 623.95 -> Flow 83087.0
Sequence of Floats: (Elev -> Flow) = [(599.95, 0.0), (600.95, 1622.0), (601.95, 3294.0), (602.95, 4888.0), (603.95, 6644.0), (604.95, 8602.0), (605.95, 10702.0), (606.95, 12800.0), (607.95, 15172.0), (608.95, 17751.0), (609.95, 20593.0), (610.95, 23651.0), (611.95, 26843.0), (612.95, 30538.0), (613.95, 34588.0), (614.95, 38599.0), (615.95, 43020.0), (616.95, 47788.0), (617.95, 52700.0), (618.95, 57777.0), (619.95, 62933.0), (620.95, 67883.0), (621.95, 72840.0), (622.95, 77803.0), (623.95, 83087.0), (624.95, 88622.0), (625.95, 94434.0), (626.95, 100363.0), (627.95, 106818.0), (628.95, 113342.0), (629.95, 119933.0), (630.95, 126732.0), (631.95, 133575.0), (632.95, 140495.0), (633.95, 147944.0), (634.95, 155521.0), (635.95, 163408.0), (636.95, 171694.0), (637.95, 179990.0), (638.95, 188764.0), (639.95, 197676.0), (640.95, 207015.0), (641.95, 216495.0), (642.95, 226100.0), (643.95, 235545.0), (644.95, 244615.0), (645.95, 253200.0), (646.95, 261233.0), (64

### Use a PairedData Object with 1 Indpendent Parameter and 2 Dependent Parameters

In [3]:
with DssDataStore.open(dss_file_name) as dss:
    pd = cast(
        PairedData,
        dss.retrieve("/BIG MUDDY SUBIMPOUNDMENT/POOL-AREA CAPACITY/ELEV-STOR-AREA////"),
    )
    elevs = pd.ind_values
    stors = pd.dep_values[0]
    areas = pd.dep_values[1]
    # ------------------- #
    # rate a single value #
    # ------------------- #
    idx = int(len(elevs) / 2)
    elev = elevs[idx]
    rated_stor = pd.rate(elev, label="stor")
    assert rated_stor == stors[idx]
    rated_area = pd.rate(elev, label="area")
    assert rated_area == areas[idx]
    print(f"Float: Elev {elev} -> Stor {rated_stor}, Area {rated_area}")
    # now reverse rate
    reverse_rated_float = pd.reverse_rate(rated_stor, label="stor")
    assert reverse_rated_float == elevs[idx]
    reverse_rated_float = pd.reverse_rate(rated_area, label="area")
    assert reverse_rated_float == elevs[idx]
    # now cross-rate
    cross_rated_float = pd.rate(pd.reverse_rate(rated_stor, label="stor"), label="area")
    assert cross_rated_float == areas[idx]
    print(f"Float: Stor {rated_stor} -> Area {cross_rated_float}")
    cross_rated_float = pd.rate(pd.reverse_rate(rated_area, label="area"), label="stor")
    assert cross_rated_float == stors[idx]
    print(f"Float: Area {rated_area} -> Stor {cross_rated_float}")
    # --------------------- #
    # rate a list of values #
    # --------------------- #
    rated_stors = pd.rate(elevs, label="stor")
    assert all([rated_stors[i] == stors[i]] for i in range(len(stors)))
    rated_areas = pd.rate(elevs, label="area")
    assert all([rated_areas[i] == stors[i]] for i in range(len(areas)))
    print(
        f"Sequence of Floats: (Elev -> Store, Area) = {list(zip(elevs, rated_stors, rated_areas))}"
    )
    # now reverse rate
    reverse_rated_sequence = pd.reverse_rate(rated_stors, label="stor")
    assert np.allclose(rated_sequence, flows, equal_nan=True)
    reverse_rated_sequence = pd.reverse_rate(rated_areas, label="area")
    assert np.allclose(reverse_rated_sequence, elevs, equal_nan=True)
    # now cross-rate
    cross_rated_sequence = pd.rate(
        pd.reverse_rate(rated_stors, label="stor"), label="area"
    )
    assert np.allclose(cross_rated_sequence, areas, equal_nan=True)
    print(f"Sequence of Floats (Stor -> Area) {list(zip(stors, cross_rated_sequence))}")
    cross_rated_sequence = pd.rate(
        pd.reverse_rate(rated_areas, label="area"), label="stor"
    )
    assert np.allclose(cross_rated_sequence, stors, equal_nan=True)
    print(
        f"Sequence of Floats (Area -> Store) {list(zip(areas, cross_rated_sequence))}"
    )
    # ------------------ #
    # rate a time series #
    # ------------------ #
    elev_ts = cast(
        TimeSeries,
        dss.retrieve("/BIG MUDDY SUBIMPOUNDMENT/POOL-AREA CAPACITY/ELEV//1Hour/Test/"),
    )
    stor_ts = cast(
        TimeSeries,
        dss.retrieve("/BIG MUDDY SUBIMPOUNDMENT/POOL-AREA CAPACITY/STOR//1Hour/Test/"),
    )
    area_ts = cast(
        TimeSeries,
        dss.retrieve("/BIG MUDDY SUBIMPOUNDMENT/POOL-AREA CAPACITY/AREA//1Hour/Test/"),
    )
    rated_stor_ts = pd.rate(elev_ts, label="stor")
    # don't compare first and last value in time series because they are out of bounds (Java extrapolates and python set to NaN)
    assert np.allclose(rated_stor_ts.values[1:-1], stor_ts.values[1:-1], equal_nan=True)
    rated_area_ts = pd.rate(elev_ts, label="area")
    assert np.allclose(rated_area_ts.values[1:-1], area_ts.values[1:-1], equal_nan=True)
    print(
        f"TimeSeries: (Elev -> Stor, Area) = {list(zip(elev_ts.values, rounder.round_f(rated_stor_ts.values), rounder.round_f(rated_area_ts.values)))}"
    )
    # now reverse rate
    reverse_rated_ts = pd.reverse_rate(rated_stor_ts, label="stor")
    assert np.allclose(
        reverse_rated_ts.values[1:-1], elev_ts.values[1:-1], equal_nan=True
    )
    reverse_rated_ts = pd.reverse_rate(rated_area_ts, label="area")
    assert np.allclose(
        reverse_rated_ts.values[1:-1], elev_ts.values[1:-1], equal_nan=True
    )
    # now cross-rate
    cross_rated_ts = pd.rate(pd.reverse_rate(stor_ts, label="stor"), label="area")
    assert np.allclose(
        cross_rated_ts.values[1:-1], area_ts.values[1:-1], equal_nan=True
    )
    print(
        f"TimeSeries: (Stor -> Area) {list(zip(rounder.round_f(stor_ts.values[1:-1]), rounder.round_f(cross_rated_ts.values[1:-1])))}"
    )
    cross_rated_ts = pd.rate(pd.reverse_rate(area_ts, label="area"), label="stor")
    assert np.allclose(
        cross_rated_ts.values[1:-1], stor_ts.values[1:-1], equal_nan=True
    )
    print(
        f"TimeSeries: (Area -> Stor) {list(zip(rounder.round_f(area_ts.values[1:-1]), rounder.round_f(cross_rated_ts.values[1:-1])))}"
    )

Float: Elev 411.69 -> Stor 11036.0, Area 833.0
Float: Stor 11036.0 -> Area 833.0
Float: Area 833.0 -> Stor 11036.0
Sequence of Floats: (Elev -> Store, Area) = [(390.0, 0.0, 0.0), (399.69, 33.0, 5.0), (400.69, 446.0, 60.0), (401.69, 860.0, 109.0), (402.69, 1273.0, 152.0), (403.69, 1687.0, 191.0), (404.69, 2100.0, 226.0), (405.69, 3056.0, 310.0), (406.69, 4011.0, 385.0), (407.69, 4967.0, 453.0), (408.69, 5922.0, 514.0), (409.69, 6878.0, 571.0), (410.69, 8811.0, 696.0), (411.69, 11036.0, 833.0), (412.69, 13566.0, 981.0), (413.69, 16416.0, 1140.0), (414.69, 19600.0, 1313.0), (415.69, 22282.0, 1442.0), (416.69, 25095.0, 1629.0), (417.69, 28032.0, 1830.0), (418.69, 31086.0, 2046.0), (419.69, 34250.0, 2261.0), (420.69, 37810.0, 2457.0), (421.69, 41524.0, 2662.0), (422.69, 45389.0, 2877.0), (423.69, 49402.0, 3102.0)]
Sequence of Floats (Stor -> Area) [(0.0, 0.0), (33.0, 5.0), (446.0, 60.0), (860.0, 109.0), (1273.0, 152.0), (1687.0, 191.0), (2100.0, 226.0), (3056.0, 310.0), (4011.0, 385.0), (49

u:\Devl\git\hec-python-library\hec\datastore.py:1489: UserWarning: HEC-DSS record unit 'undef' is invalid for base parameter 'STOR', used 'ac-ft' instead
  warnings.warn(
u:\Devl\git\hec-python-library\hec\datastore.py:1489: UserWarning: HEC-DSS record unit 'undef' is invalid for base parameter 'AREA', used 'ft2' instead
  warnings.warn(


TimeSeries: (Elev -> Stor, Area) = [(385.155, nan, nan), (390.0, 0.0, 0.0), (394.845, 16.5, 2.5), (399.69, 33.0, 5.0), (400.19, 239.5, 32.5), (400.69, 446.0, 60.0), (401.19, 653.0, 84.5), (401.69, 860.0, 109.0), (402.19, 1066.5, 130.5), (402.69, 1273.0, 152.0), (403.19, 1480.0, 171.5), (403.69, 1687.0, 191.0), (404.19, 1893.5, 208.5), (404.69, 2100.0, 226.0), (405.19, 2578.0, 268.0), (405.69, 3056.0, 310.0), (406.19, 3533.5, 347.5), (406.69, 4011.0, 385.0), (407.19, 4489.0, 419.0), (407.69, 4967.0, 453.0), (408.19, 5444.5, 483.5), (408.69, 5922.0, 514.0), (409.19, 6400.0, 542.5), (409.69, 6878.0, 571.0), (410.19, 7844.5, 633.5), (410.69, 8811.0, 696.0), (411.19, 9923.5, 764.5), (411.69, 11036.0, 833.0), (412.19, 12301.0, 907.0), (412.69, 13566.0, 981.0), (413.19, 14991.0, 1060.5), (413.69, 16416.0, 1140.0), (414.19, 18008.0, 1226.5), (414.69, 19600.0, 1313.0), (415.19, 20941.0, 1377.5), (415.69, 22282.0, 1442.0), (416.19, 23688.5, 1535.5), (416.69, 25095.0, 1629.0), (417.19, 26563.5, 1

### Use PairedData Object with 2 Independent Parameters and 1 Dependent Parameter

In [4]:
with DssDataStore.open(dss_file_name) as dss:
    pd = cast(
        PairedData,
        dss.retrieve(
            "/Lake Shelbyville/Sluices-Gate Rating/Elev-Flow/PairedValuesExt///"
        ),
    )
    elevs = pd.ind_values
    openings = list(map(float, cast(list[str], pd.labels)))
    flows2D = pd.dep_values
    idx1 = int(len(elevs) / 2)
    elev = elevs[idx1]
    idx2 = int(len(openings) / 2)
    opening = openings[idx2]
    # ----------------------- #
    # rate a single value set #
    # ----------------------- #
    rated_flow = pd.rate([[elev, opening]])[0]
    assert rated_flow == flows2D[idx2][idx1]
    print(
        f"Value Set: Elev {rounder.round_f([elev])[0]}, Opening {rounder.round_f([opening])[0]} -> flow {rounder.round_f([rated_flow])[0]}"
    )
    # ------------------------- #
    # rate a list of value sets #
    # ------------------------- #
    rated_sequence = pd.rate(PairedData.transform(elevs, len(elevs) * [opening]))
    assert np.allclose(rated_sequence, flows2D[idx2])
    print(
        f"Sequence of Value Sets: (Elev, Opening) -> Flow {list(zip(list(zip(rounder.round_f(elevs), len(elevs) * [opening])), rounder.round_f(rated_sequence)))}"
    )
    # ------------------------- #
    # rate a set of time series #
    # ------------------------- #
    elev_ts = cast(
        TimeSeries,
        dss.retrieve("/Lake Shelbyville/Sluices-Gate Rating/Elev//1Hour/Test/"),
    )
    opening_ts = cast(
        TimeSeries,
        dss.retrieve(
            "/Lake Shelbyville/Sluices-Gate Rating/Opening//1Hour/Test-5.500000/"
        ),
    )
    flow_ts = cast(
        TimeSeries,
        dss.retrieve(
            "/Lake Shelbyville/Sluices-Gate Rating/Flow//1Hour/Test-5.500000/"
        ),
    )
    rated_ts = pd.rate([elev_ts, opening_ts])
    # don't compare first and last value in time series because they are out of bounds (Java extrapolates and python set to NaN)
    # also, the rated time series from the HEC-DSS file (rated using hec.hecmath.PairedDataMath objects) are one item shorter than expected
    assert np.allclose(rated_ts.values[1:-2], flow_ts.values[1:-1], equal_nan=True)
    print(
        f"Time Series Set: (Elev, Opening) -> Flow {list(zip(list(zip(rounder.round_f(elev_ts.values), (len(elev_ts)-1) * [opening])), rounder.round_f(rated_ts.values)))}"
    )

Value Set: Elev 605.7, Opening 5.5 -> flow 1332.18
Sequence of Value Sets: (Elev, Opening) -> Flow [((565.7, 5.5), 512.0), ((566.7, 5.5), 544.26), ((567.7, 5.5), 575.65), ((568.7, 5.5), 606.18), ((569.7, 5.5), 635.87), ((570.7, 5.5), 664.76), ((571.7, 5.5), 692.86), ((572.7, 5.5), 720.19), ((573.7, 5.5), 746.77), ((574.7, 5.5), 772.63), ((575.7, 5.5), 797.78), ((576.7, 5.5), 822.25), ((577.7, 5.5), 846.05), ((578.7, 5.5), 869.2), ((579.7, 5.5), 891.72), ((580.7, 5.5), 913.63), ((581.7, 5.5), 934.93), ((582.7, 5.5), 955.66), ((583.7, 5.5), 975.82), ((584.7, 5.5), 995.43), ((585.7, 5.5), 1014.5), ((586.7, 5.5), 1033.06), ((587.7, 5.5), 1051.11), ((588.7, 5.5), 1068.66), ((589.7, 5.5), 1085.74), ((590.7, 5.5), 1102.35), ((591.7, 5.5), 1118.62), ((592.7, 5.5), 1134.89), ((593.7, 5.5), 1151.16), ((594.7, 5.5), 1167.35), ((595.7, 5.5), 1182.8), ((596.7, 5.5), 1198.24), ((597.7, 5.5), 1213.69), ((598.7, 5.5), 1229.13), ((599.7, 5.5), 1244.57), ((600.7, 5.5), 1259.36), ((601.7, 5.5), 1274.12),

u:\Devl\git\hec-python-library\hec\datastore.py:1489: UserWarning: HEC-DSS record unit 'cfs' is invalid for base parameter 'Opening', used 'ft' instead
  warnings.warn(


Time Series Set: (Elev, Opening) -> Flow [((566.2, 5.5), 528.13), ((565.7, 5.5), 512.0), ((566.2, 5.5), 528.13), ((566.7, 5.5), 544.26), ((567.2, 5.5), 559.955), ((567.7, 5.5), 575.65), ((568.2, 5.5), 590.915), ((568.7, 5.5), 606.18), ((569.2, 5.5), 621.025), ((569.7, 5.5), 635.87), ((570.2, 5.5), 650.315), ((570.7, 5.5), 664.76), ((571.2, 5.5), 678.81), ((571.7, 5.5), 692.86), ((572.2, 5.5), 706.525), ((572.7, 5.5), 720.19), ((573.2, 5.5), 733.48), ((573.7, 5.5), 746.77), ((574.2, 5.5), 759.7), ((574.7, 5.5), 772.63), ((575.2, 5.5), 785.205), ((575.7, 5.5), 797.78), ((576.2, 5.5), 810.015), ((576.7, 5.5), 822.25), ((577.2, 5.5), 834.15), ((577.7, 5.5), 846.05), ((578.2, 5.5), 857.625), ((578.7, 5.5), 869.2), ((579.2, 5.5), 880.46), ((579.7, 5.5), 891.72), ((580.2, 5.5), 902.675), ((580.7, 5.5), 913.63), ((581.2, 5.5), 924.28), ((581.7, 5.5), 934.93), ((582.2, 5.5), 945.295), ((582.7, 5.5), 955.66), ((583.2, 5.5), 965.74), ((583.7, 5.5), 975.82), ((584.2, 5.5), 985.625), ((584.7, 5.5),

### Use a PairedData Object with 1 Indpendent Parameter, 1 Dependent Parameter and Multiple Depdendent Sub-parameters

In [5]:
with DssDataStore.open(dss_file_name) as dss:
    # --------------------------------------------------------------------------------- #
    # Convert a PairedData with 2 independent and 1 dependent parameter to a PairedData #
    # #object with 1 independent and 1 dependent parameter with multiple sub-parameters #
    # --------------------------------------------------------------------------------- #
    pd = cast(
        PairedData,
        dss.retrieve(
            "/Lake Shelbyville/Sluices-Gate Rating/Elev-Flow/PairedValuesExt///"
        ),
    )
    info = json.loads(pd.json)
    # convert labels to non-numeric
    info["labels"] = [f"Opening_{lbl}" for lbl in info["labels"]]
    pd = PairedData(json.dumps(info))
    elevs = pd.ind_values
    flows2D = pd.dep_values
    idx1 = int(len(elevs) / 2)
    sub_param = "Opening_5.5"
    idx2 = cast(list[str], pd.labels).index(sub_param)
    # ------------------- #
    # rate a single value #
    # ------------------- #
    elev = elevs[idx1]
    rated_float = pd.rate(elev, label=sub_param)
    assert rated_float == flows2D[idx2][idx1]
    print(f"Float: Elev {elev} -> Flow {rated_float}")
    # now reverse rate
    reverse_rated_float = pd.reverse_rate(rated_float, label=sub_param)
    assert reverse_rated_float == elev
    # --------------------- #
    # rate a list of values #
    # --------------------- #
    rated_sequence = pd.rate(elevs, label=sub_param)
    assert np.allclose(rated_sequence, flows2D[idx2], equal_nan=True)
    print(f"Sequence of Floats: (Elev -> Flow) = {list(zip(elevs, rated_sequence))}")
    # now reverse rate
    reverse_rated_sequence = pd.reverse_rate(rated_sequence, label=sub_param)
    assert np.allclose(reverse_rated_sequence, elevs, equal_nan=True)
    # ------------------ #
    # rate a time series #
    # ------------------ #
    elev_ts = cast(
        TimeSeries,
        dss.retrieve("/Lake Shelbyville/Sluices-Gate Rating/Elev//1Hour/Test/"),
    )
    flow_ts = cast(
        TimeSeries,
        dss.retrieve(
            "/Lake Shelbyville/Sluices-Gate Rating/Flow//1Hour/Test-5.500000/"
        ),
    )
    rated_ts = pd.rate(elev_ts, label=sub_param)
    # don't compare first and last value in time series because they are out of bounds (Java extrapolates and python set to NaN)
    # also, the rated time series from the HEC-DSS file (rated using hec.hecmath.PairedDataMath objects) are one item shorter than expected
    assert np.allclose(rated_ts.values[1:-2], flow_ts.values[1:-1], equal_nan=True)
    print(f"TimeSeries: (Elev -> Flow) = {list(zip(elev_ts.values, rated_ts.values))}")

Float: Elev 605.7000122070312 -> Flow 1332.1800537109375
Sequence of Floats: (Elev -> Flow) = [(565.7000122070312, 512.0), (566.7000122070312, 544.260009765625), (567.7000122070312, 575.6500244140625), (568.7000122070312, 606.1799926757812), (569.7000122070312, 635.8699951171875), (570.7000122070312, 664.760009765625), (571.7000122070312, 692.8599853515625), (572.7000122070312, 720.1900024414062), (573.7000122070312, 746.77001953125), (574.7000122070312, 772.6300048828125), (575.7000122070312, 797.780029296875), (576.7000122070312, 822.25), (577.7000122070312, 846.0499877929688), (578.7000122070312, 869.2000122070312), (579.7000122070312, 891.719970703125), (580.7000122070312, 913.6300048828125), (581.7000122070312, 934.9299926757812), (582.7000122070312, 955.6599731445312), (583.7000122070312, 975.8200073242188), (584.7000122070312, 995.4299926757812), (585.7000122070312, 1014.5), (586.7000122070312, 1033.06005859375), (587.7000122070312, 1051.1099853515625), (588.7000122070312, 1068.